In [ ]:
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

from flask import Flask, jsonify

In [ ]:
engine = create_engine("sqlite:///hawaii.sqlite")

In [ ]:
session = Session(bind=engine)

In [ ]:
Base = automap_base()
Base.prepare(engine, reflect=True)
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
results = session.query(Measurement.tobs).all()

In [ ]:
tobs_values = list(np.ravel(results))

In [ ]:
# Query for last 12 months of precipitation
precipitation = session.query(Measurement.date, Measurement.prcp).\
        filter(Measurement.date >= '2017-04-25').filter(Measurement.date <= '2018-04-25').order_by(Measurement.date).all()

In [ ]:
precipitation_df = pd.DataFrame(data=precipitation)
precipitation_df.head()

In [ ]:
precipitation_df = precipitation_df.set_index("date")
precipitation_df.head()

In [ ]:
#plot precipitation data

# Define labels
plt.title("Precipitation for last 12 Months")
plt.xlabel("Month")
plt.ylabel("Precipitation in inches")

# Define months for x-ticks labels
months = ["Apr", "May", "Jun", "Jul", "Aug", "Sept", "Oct", "Nov", "Dec", "Jan", "Feb", "Mar"]

# Define X and Y values
y = precipitation_df["prcp"].tolist()
x = np.arange(0, len(precipitation_df.index.tolist()), 1)

plt.bar(x, y, width=30, color="blue", alpha=0.5, align="edge")

plt.show()

In [ ]:
precipitation_df.describe()

In [ ]:
#Station Analysis
num_stations = session.query(func.count(Station.station)).first()

In [ ]:
print(f"Total number of stations: {str(num_stations[0])}")

In [ ]:
#most active stations
engine.execute("SELECT count(station), station FROM measurement GROUP BY station ORDER BY count(station) DESC").fetchall()

In [ ]:
active_stations_desc = session.query(Measurement.station, func.count(Measurement.station)).\
        group_by(Measurement.station).order_by(func.count(Measurement.station).desc()).all()

In [ ]:
active_stations_desc_df = pd.DataFrame(data=active_stations_desc, columns=['Station', 'Count'])
active_stations_desc_df.head()

In [ ]:
station_most = active_stations_desc_df["Station"][0]
most_observations = active_stations_desc_df["Count"][0]
print(f"Station with most observations ({most_observations}): {station_with_most_observations}")

In [ ]:
temperature_freq = session.query(Measurement.tobs).\
    filter(Measurement.date >= '2017-04-25').\
    filter(Measurement.station == station_most).\
    order_by(Measurement.tobs).all()

In [ ]:
# Define the histogram from the above dataset, with bins=12
hist, bins = np.histogram(temperature_freq, bins=12)
width = bins[1] - bins[0]
plt.bar(hist, width=width)
plt.show()

In [ ]:
#calc temps
def calc_temps(start_date, end_date):
     return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

In [ ]:
def last_year(start_date, end_date):
    lst_start_date = start_date.split('-')
    lst_end_date = end_date.split('-')
    last_year_start = int(lst_start_date[0]) - 1
    last_year_end = int(lst_end_date[0]) - 1
    ly_start_date = f"{last_year_start}-{lst_start_date[1]}-{lst_start_date[2]}"
    ly_end_date = f"{last_year_end}-{lst_end_date[1]}-{lst_end_date[2]}"
    
    return (ly_start_date, ly_end_date)

In [ ]:
trip_start = '2018-04-15'
trip_end = '2018-04-25'

In [ ]:
avg_trip_temps = calc_temps(trip_start, trip_end)
(last_year_start, last_year_end) = last_year(trip_start, trip_end)

In [ ]:
yerr_val = avg_trip_temps[0][2] - avg_trip_temps[0][0]
y = [average_trip_temps[0][1]]
x = 0

In [ ]:
# Define plot
fig, ax = plt.subplots()
ax.set_ylabel("Temperature (F)")
ax.set_title("Trip Average Temps")

ax.bar(x, y, width=.1, color="blue", yerr=yerr_val)
ax.set_xlim(-.1, .1)
ax.set_ylim(0, 100)
plt.show()

In [ ]:
#Flask App
import sqlalchemy
import numpy as np
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from flask import Flask, jsonify

app = Flask(__name__)

In [ ]:
engine = create_engine("sqlite:///hawaii.sqlite")

In [ ]:
Base = automap_base()

In [ ]:
Base.prepare(engine, reflect=True)

In [ ]:
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
session = Session(engine)

In [ ]:
@app.route("/api/v1.0/precipitation")
def precipitation():
 #Return a list of measurement date and prcp information from the last year
    results = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= '2017-04-25').order_by(Measurement.date)

In [ ]:
    precipitation_values = []
    for p in results:
        prcp_dict = {}
        prcp_dict["date"] = p.date
        prcp_dict["prcp"] = p.prcp
        precipitation_values.append(prcp_dict)

    return jsonify(precipitation_values)

In [ ]:
@app.route("/api/v1.0/stations")
def stations():
    #Return a list of all station names
    # Query all stations
    results = session.query(Station.name).all()
    
    station_names = list(np.ravel(results))

    return jsonify(station_names)

In [ ]:
# Return a json list of Temperature Observations (tobs) for the previous year
@app.route("/api/v1.0/tobs")
def tobs():
    results = session.query(Measurement.tobs).all()
    tobs_values = list(np.ravel(results))

    return jsonify(tobs_values)

In [ ]:
# Return a json list of the minimum temperature, the average temperature, and the max 
# temperature for a given start or start-end range.
@app.route("/api/v1.0/<start>")
def temperatures_start(start):
    results = session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
                filter(Measurement.date >= start).all()
        temperatures_start = list(np.ravel(results))

    return jsonify(temperatures_start)

In [ ]:
@app.route("/api/v1.0/<start>/<end>")
def temperatures_start_end(start, end):
    results = session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
                filter(Measurement.date >= start).\
                filter(Measurement.date <= end).all()
            temperatures_start_end = list(np.ravel(results))

    return jsonify(temperatures_start_end)